In [ ]:
# TODO: Change the values of these.
# Define range of red color in HSV space
lower_red1 = (0, 0, 0)
upper_red1 = (0, 0, 0)
lower_red2 = (0, 0, 0)
upper_red2 = (0, 0, 0)

# Minimum area we look at
# Optional: Change this as needed
min_area = 800

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import time
import threading

stop_event = threading.Event()

# Initialize Instances from our Packages (Only run once)
robot = Robot()
camera = Camera.instance()

# Function of Image Processing
def update_loop():
    global robot_enabled
    global robot_stopsign_detected
    global recent_detection
    while not stop_event.is_set():
        if robot_enabled:
            if robot_stopsign_detected:
                robot.stop()
            elif (time.time() - recent_detection) > 0.5:
                robot.forward(0.1)
        else:
            robot.stop()
        clear_output(wait = True)
        orig_image = np.array(camera.value)
        image = orig_image

        # Convert to HSV (Hue, Saturation, Value) color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Create a mask for red color
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = mask1 | mask2

        # Apply the mask to the image
        red_traffic_signs = cv2.bitwise_and(image, image, mask=mask)

        # Dilate image to get rid of white stop text
        
        dilated_image = cv2.dilate(red_traffic_signs, np.ones((3, 3), np.uint8), iterations = 3)
        eroded_image = cv2.erode(dilated_image, np.ones((5, 5), np.uint8))

        # Convert to grayscale
        gray = cv2.cvtColor(eroded_image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect edges using Canny edge detector
        edges = cv2.Canny(blurred, 100, 200)

        # Find contours
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Loop through contours
        for contour in contours:
            contour_area = cv2.contourArea(contour)
            if contour_area > min_area:
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * (contour_area / (perimeter * perimeter))
                # Approximate the contour to a polygon
                epsilon = 0.01 * perimeter
                approx = cv2.approxPolyDP(contour, epsilon, True)
                
                vertices_label.value = f"Number of Vertices: {len(approx)}"
                circularity_label.value = f"Circularity: {circularity:.2f}"

                # If the shape has 8 vertices, it could be a stop sign (octagon)
                if len(approx) >= 5 and len(approx) <= 13 and circularity > 0.7 and circularity < 1.3:
                    cv2.drawContours(orig_image, [approx], -1, (0, 255, 0), 2)
                    cv2.putText(orig_image, 'Stop Sign', tuple(approx[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    cv2.putText(orig_image, str(contour_area), (approx[0][0][0], approx[0][0][1] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    if contour_area > 1000:
                        robot.stop()
                        recent_detection = time.time()
                        robot_stopsign_detected = True
                    else:
                        robot_stopsign_detected = False
            else:
                robot_stopsign_detected = False
        
        overlay_widget.value = bgr8_to_jpeg(orig_image)
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR))
        
        time.sleep(0.05)

# Display Block
# Image Debugging Widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
vertices_label = widgets.Label(value = "Number of Vertices:")
circularity_label = widgets.Label(value = "Circularity:")
# Start and stop buttons to control robot
go_button = widgets.Button(description="Go", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')
robot_enabled = False  # flag to control robot movement
robot_stopsign_detected = False
recent_detection = 0

# Button Callbacks
def on_go_clicked(b):
    global robot_enabled
    robot_enabled = True

def on_stop_clicked(b):
    global robot_enabled
    robot_enabled = False
    robot.stop()

go_button.on_click(on_go_clicked)
stop_button.on_click(on_stop_clicked)

display(widgets.VBox([widgets.HBox([widgets.Label("Overlay"), widgets.Label("Edge Mask"),]),
                      widgets.HBox([overlay_widget, edge_mask_widget]),
                      widgets.HBox([go_button, stop_button]),
                      vertices_label,
                      circularity_label]
                    ))

camera.start()
thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()